#### SQLite port_lite database: sales, stocks table
#### csv files: daily-watch-list, daily-sales, monitors, stocks-all

### Restart and Run All Cells

In [1]:
import numpy as np
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 12, 21)

### Yesterday = Last business day

In [2]:
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2022-12-21
yesterday: 2022-12-20


In [3]:
cols = 'fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

### Begin of Tables in the process

In [4]:
sql = """
SELECT * 
FROM sales 
ORDER BY id DESC LIMIT 1
"""
tmp = pd.read_sql(sql, conlite)
tmp["fm_date"] = pd.to_datetime(tmp["fm_date"])
tmp["to_date"] = pd.to_datetime(tmp["to_date"])
tmp["created_at"] = pd.to_datetime(tmp["created_at"])
tmp["updated_at"] = pd.to_datetime(tmp["updated_at"])
tmp.style.format(format_dict)

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1196028,DIF,2022-12-19,2022-12-20,1,13.20,13.10,-0.10,-0.76%,-1,-1,13.20,13.10,"1,736,699",2022-12-20,2022-12-20,1


In [5]:
names = tmp["name"]
type(names)

pandas.core.series.Series

In [6]:
name = names.to_string(index=False)
type(name)

str

In [7]:
sql = """
SELECT * 
FROM stocks
WHERE name = '%s'
"""
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * 
FROM stocks
WHERE name = 'DIF'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,325,DIF,14.50,11.70,S,13.20,14.60,147.37,0.39,14.70,"10,000",-5,5,"40,000",0,0,CUT,SET


### End of Tables in the process

In [8]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2022-12-20' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [9]:
df = pd.read_sql(sql, conlite)
df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df['diff'] = (df['to_price'] - df['fm_price'])
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
name,,,,,,,,,,,
RCL,2022-12-19,2022-12-20,30.00,29.50,"2,774,362",30.25,29.50,B,SET100,-0.50,-1.67%
STA,2022-12-16,2022-12-20,19.10,18.50,"6,973,011",19.10,-0.52,B,SET100,-0.60,-3.14%
SYNEX,2022-12-19,2022-12-20,15.90,15.60,"2,129,233",16.00,15.50,B,SET100,-0.30,-1.89%
DOHOME,2022-12-19,2022-12-20,14.20,14.10,"14,835,312",14.50,13.90,I,SET100,-0.10,-0.70%
JASIF,2022-12-19,2022-12-20,8.00,8.05,"2,280,060",8.05,8.00,I,SET,0.05,0.63%
MCS,2022-12-16,2022-12-20,8.95,8.85,"1,668,340",8.90,-1.12,I,SET,-0.10,-1.12%
SCC,2022-12-19,2022-12-20,331.00,332.00,"2,421,904",332.00,328.00,I,SET50,1.00,0.30%
BH,2022-12-16,2022-12-20,214.00,209.00,"3,719,849",212.00,-0.93,O,SET50,-5.00,-2.34%
CKP,2022-12-19,2022-12-20,4.78,4.66,"19,156,474",4.76,4.62,O,SET100,-0.12,-2.51%


### Price change up or down more than 3%

In [10]:
# For each row in the DataFrame, generate a series of dates between the start and end dates
df['dates'] = df.apply(lambda x: pd.date_range(x['fm_date'], x['to_date']), axis=1)

# For each row in the DataFrame, select only the business days (Monday through Friday) from the series of dates
df['business_days'] = df['dates'].apply(lambda x: x[x.isin(pd.date_range(x.min(), x.max(), freq='B'))])

# For each row in the DataFrame, count the number of business days between the start and end dates
df['bdays'] = df['business_days'].apply(lambda x: len(x))

In [11]:
colt =  'percent status fm_price to_price market \
fm_date bdays max_price min_price'.split()
watch_list = df.loc[abs(df.percent) >= 3,colt]
watch_list.sort_values(by="percent",ascending=False).style.format(format_dict)

,percent,status,fm_price,to_price,market,fm_date,bdays,max_price,min_price
name,,,,,,,,,
SINGER,6.36%,O,27.50,29.25,SET100,2022-12-16,3,30.50,6.36
III,4.58%,O,13.10,13.70,SET,2022-12-09,8,13.80,4.58
TMT,4.32%,S,6.95,7.25,SET,2022-12-09,8,7.25,2.16
GFPT,3.15%,O,12.70,13.10,SET,2022-12-15,4,13.20,3.15
STA,-3.14%,B,19.10,18.50,SET100,2022-12-16,3,19.10,-0.52
MEGA,-4.28%,O,46.75,44.75,SET100,2022-12-13,6,45.75,-3.21


In [12]:
file_name = "daily-watch-list.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

watch_list.sort_values(['percent'],ascending=[False]).to_csv(output_file)
watch_list.sort_values(['percent'],ascending=[False]).to_csv(data_file)
watch_list.sort_values(['percent'],ascending=[False]).to_csv(box_file)
watch_list.sort_values(['percent'],ascending=[False]).to_csv(one_file)

### Create daily-sales from sales

In [13]:
df[cols].style.format(format_dict)

,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
name,,,,,,,,,
RCL,2022-12-19,2022-12-20,30.00,29.50,"2,774,362",30.25,29.50,-1.67%,B
STA,2022-12-16,2022-12-20,19.10,18.50,"6,973,011",19.10,-0.52,-3.14%,B
SYNEX,2022-12-19,2022-12-20,15.90,15.60,"2,129,233",16.00,15.50,-1.89%,B
DOHOME,2022-12-19,2022-12-20,14.20,14.10,"14,835,312",14.50,13.90,-0.70%,I
JASIF,2022-12-19,2022-12-20,8.00,8.05,"2,280,060",8.05,8.00,0.63%,I
MCS,2022-12-16,2022-12-20,8.95,8.85,"1,668,340",8.90,-1.12,-1.12%,I
SCC,2022-12-19,2022-12-20,331.00,332.00,"2,421,904",332.00,328.00,0.30%,I
BH,2022-12-16,2022-12-20,214.00,209.00,"3,719,849",212.00,-0.93,-2.34%,O
CKP,2022-12-19,2022-12-20,4.78,4.66,"19,156,474",4.76,4.62,-2.51%,O


In [14]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file,header=None)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(data_file,header=None)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(box_file)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(one_file)

### Create monitors from orders

In [15]:
sql = """
SELECT name, trade 
FROM orders 
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors.sample(5)

,trade
name,
JMART,B
DIF,S
JASIF,S
ASP,S
MEGA,B


In [16]:
sql = """
SELECT trade, COUNT(*) AS 'no. of stocks'
FROM orders 
GROUP BY trade
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,trade,no. of stocks
0,B,14
1,S,7


### output_file = iCloud Drive on PC

In [17]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

monitors.to_csv(data_file, header=None)
monitors.to_csv(box_file)
monitors.to_csv(output_file)
monitors.to_csv(one_file)

### End of daily process

In [18]:
sql = """
SELECT trade, COUNT(*) AS 'no. of stocks'
FROM orders 
GROUP BY trade
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,trade,no. of stocks
0,B,14
1,S,7


In [19]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,status,market
name,,
RCL,B,SET100
STA,B,SET100
SYNEX,B,SET100
DOHOME,I,SET100
JASIF,I,SET
...,...,...
SC,X,SET
SIRI,X,SET
SPALI,X,SET100


In [20]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(box_file)
stocks.to_csv(one_file)

In [21]:
sql = """
SELECT name, trade 
FROM orders 
WHERE trade IN ("B")
ORDER BY name
"""
buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,trade
name,
BH,B
CKP,B
CPF,B
GFPT,B
III,B
JMART,B
KCE,B
MEGA,B
RCL,B


In [22]:
buy_candidates.shape[0]

14

In [23]:
sql = """
SELECT name, trade 
FROM orders 
WHERE trade IN ("S")
ORDER BY name
"""
sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,trade
name,
ASP,S
DIF,S
DOHOME,S
JASIF,S
MCS,S
SCC,S
TMT,S


In [24]:
sell_candidates.shape[0]

7